# YOLO - Training
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jul 2021<br>
**Descripción:** Notebook de entrenamiento.<br>

## Instalación de OpenCV

``` bash
    sudo apt install python3-opencv
    sudo apt-get install libopencv-dev
```

## Instalar zip

``` bash
    sudo apt install zip unzip
```

Uso

``` bash
    zip -r nombre_comprimido.zip directorio_a_comprimir
    unzip nombre_comprimido.zip -d directorio_a_descomprimir
``` 

## Preparación del entorno

* Crear entorno virtual (solo la primera vez) dentro de la carpeta `/training/<version de entrena>`.

``` bash
    python3 -m venv yolo_training --system-site-packages # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source yolo_training/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r env # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

* Agregar kernel a JupyterLab (**con el entorno virtual activado**).
    
``` bash
    python -m ipykernel install --user --name=yolo_training 
```

## Clonación y construcción de Darknet (por única vez)

In [1]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15287, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15287 (delta 0), reused 1 (delta 0), pack-reused 15283
Receiving objects: 100% (15287/15287), 13.69 MiB | 23.76 MiB/s, done.
Resolving deltas: 100% (10377/10377), done.


In [2]:
%cd darknet

/home/maximiliano/ml_uali/training/yolo_v2/darknet


### Localmente

In [ ]:
# cambie el makefile para tener GPU y OPENCV habilitados en el caso necesario y verificar los paths
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

### Server

Copiar Makefile_mod a Darknet/ modificando nombre a Makefile (solo en el server)

### Compilar

In [ ]:
# make darknet (crea darknet para que luego pueda usar el archivo ejecutable darknet para ejecutar o entrenar detectores de objetos)
!make -d

In [3]:
!cat Makefile
!make

GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)
# set ZED_CAMERA=1 to enable ZED SDK 3.0 and above
# set ZED_CAMERA_v2_8=1 to enable ZED SDK 2.X

USE_CPP=0
DEBUG=0

ARCH= -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	    -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# GeForce RTX 3070, 3080, 3090
# ARCH= -gencode arch=compute_86,code=[sm_86,compute_86]

# Kepler GeForce GTX 770, GTX 760, GT 740
# ARCH= -gencode arch=compute_30,code=sm_30

# Tesla A100 (GA100), DGX-A100, RTX 3080
# ARCH= -gencode arch=compute_80,code=[sm_80,compute_80]

# Tesla V100
# ARCH= -gencode arch=compute_7

## Definir funciones auxiliares

In [4]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## Entrenamiento personalizado
### Descargando conjunto de datos por URL

**Visdrone**

obj_data URL: https://drive.google.com/file/d/1mAN5DfuuYn94tIy5pTAMryRB6JR4AaLA/view?usp=sharing

test_data URL: https://drive.google.com/file/d/10MY9BbGyfZMHnzVV5r9qYi9wJagqtkVA/view?usp=sharing

readme URL: https://drive.google.com/file/d/1cubZwBp4p_keMsZc6Ne-LgDWZsuYKxrR/view?usp=sharing

In [6]:
%cd ..

/home/maximiliano/ml_uali/training/yolo_v2


In [8]:
# obj_data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mAN5DfuuYn94tIy5pTAMryRB6JR4AaLA' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1mAN5DfuuYn94tIy5pTAMryRB6JR4AaLA" -O obj.zip && rm -rf /tmp/cookies.txt
# test_data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10MY9BbGyfZMHnzVV5r9qYi9wJagqtkVA' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10MY9BbGyfZMHnzVV5r9qYi9wJagqtkVA" -O test.zip && rm -rf /tmp/cookies.txt

--2021-07-29 11:28:12--  https://docs.google.com/uc?export=download&confirm=bN8w&id=1mAN5DfuuYn94tIy5pTAMryRB6JR4AaLA
Resolving docs.google.com (docs.google.com)... 142.250.138.138, 142.250.138.100, 142.250.138.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.138.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-5g-docs.googleusercontent.com/docs/securesc/5ckek2dqh431pcunpqlg91orsj5sl1rq/fb9use7h17dd76cv42p297o0jls2lk10/1627558050000/04394533313268812415/00086974498586315450Z/1mAN5DfuuYn94tIy5pTAMryRB6JR4AaLA?e=download [following]
--2021-07-29 11:28:12--  https://doc-0s-5g-docs.googleusercontent.com/docs/securesc/5ckek2dqh431pcunpqlg91orsj5sl1rq/fb9use7h17dd76cv42p297o0jls2lk10/1627558050000/04394533313268812415/00086974498586315450Z/1mAN5DfuuYn94tIy5pTAMryRB6JR4AaLA?e=download
Resolving doc-0s-5g-docs.googleusercontent.com (doc-0s-5g-docs.googleusercontent.com)... 142.250.115.132, 2607:f8b0:4023:1004::84
Conn

In [ ]:
# descomprima los conjuntos de datos y su contenido para que ahora estén en la carpeta /darknet/data/
!unzip obj.zip -d darknet/data/
!unzip test.zip -d darknet/data/

# eliminando carpetas
!rm -rf obj.zip
!rm -rf test.zip

Archive:  obj.zip
   creating: darknet/data/train/
  inflating: darknet/data/train/0000002_00005_d_0000014.jpg  
  inflating: darknet/data/train/0000002_00005_d_0000014.txt  
  inflating: darknet/data/train/0000002_00448_d_0000015.jpg  
  inflating: darknet/data/train/0000002_00448_d_0000015.txt  
  inflating: darknet/data/train/0000003_00231_d_0000016.jpg  
  inflating: darknet/data/train/0000003_00231_d_0000016.txt  
  inflating: darknet/data/train/0000007_04999_d_0000036.jpg  
  inflating: darknet/data/train/0000007_04999_d_0000036.txt  
  inflating: darknet/data/train/0000007_05499_d_0000037.jpg  
  inflating: darknet/data/train/0000007_05499_d_0000037.txt  
  inflating: darknet/data/train/0000007_05999_d_0000038.jpg  
  inflating: darknet/data/train/0000007_05999_d_0000038.txt  
  inflating: darknet/data/train/0000008_00889_d_0000039.jpg  
  inflating: darknet/data/train/0000008_00889_d_0000039.txt  
  inflating: darknet/data/train/0000008_01999_d_0000040.jpg  
  inflating: darkne

### Copiando archivos de configuración


In [ ]:
!unzip data_training.zip

In [ ]:
!cp data_training/obj.data darknet/data/obj.data
!cp data_training/obj.names darknet/data/obj.names
!cp data_training/yolov4-obj.cfg darknet/cfg/yolov4-obj.cfg

In [ ]:
!mkdir # aca agregar carpeta checkpoints en darknet

### Descargando archivos de configuración (Solo en Colab)


In [ ]:
# descargando obj.data, obj.names y yolov4-obj.cfg 
!wget https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training/obj.data
!wget https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training/obj.names
!wget https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training/yolov4-obj.cfg

In [ ]:
!cp obj.data darknet/data/obj.data
!cp obj.names darknet/data/obj.names
!cp yolov4-obj.cfg darknet/cfg/yolov4-obj.cfg

In [ ]:
!rm -rf obj.data
!rm -rf obj.names
!rm -rf yolov4-obj.cfg

### Generando train.txt y test.txt

In [ ]:
%cd darknet/

In [ ]:
import os

In [ ]:
# Generando test.txt

image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [ ]:
!mkdir -p data/obj

In [ ]:
# Generando train.txt

image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

### Descargue pesos pre-entrenados para las capas convolucionales.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### Entrena tu detector personalizado!

Mirar los paralametros de la instrucción ./darknet!!!!

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# muestre chart.png de cómo le fue al detector de objetos personalizado con la capacitación
imShow('chart.png')

 ### Iniciar el entrenamiento desde donde se guardó por última vez (hay que mirarlo!!!)

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/ml_uali/training/yolo_v1/checkpoints/yolov4-obj_last.weights -dont_show -map

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/ml_uali/training/yolo_v1/checkpoints/yolov4-obj_final.weights